In [58]:
import pandas as pd
import numpy as np
import os
import DataLoader
import librosa
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.metrics import mean_squared_error
from keras import backend as K
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(mean_squared_error(y_true, y_pred))

In [7]:
fullnTable = pd.read_csv("data/train.csv")

fullTable = fullnTable.sample(n=210,axis=0).reset_index(drop=True)
trainTable = fullTable[:200]
valTable = fullTable[200:]

In [103]:
t = pd.read_csv("data/train.csv")
t[t["ebird_code"]=="aldfly"][t["rating"]>=4.5]

C:\Users\karamvenkatsaigiridh\AppData\Local\Continuum\Anaconda3\lib\site-packages\ipykernel\__main__.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  from ipykernel import kernelapp as app


,rating,playback_used,ebird_code,channels,date,pitch,duration,filename,speed,species,...,xc_id,url,country,author,primary_label,longitude,length,time,recordist,license
10,4.5,no,aldfly,1 (mono),2013-06-23,Not specified,269,XC139577.mp3,Not specified,Alder Flycatcher,...,139577,https://www.xeno-canto.org/139577,United States,Jonathon Jongsma,Empidonax alnorum_Alder Flycatcher,-90.6776,Not specified,06:57,Jonathon Jongsma,Creative Commons Attribution-ShareAlike 3.0
12,5.0,no,aldfly,2 (stereo),2008-07-02,Not specified,19,XC142065.mp3,Not specified,Alder Flycatcher,...,142065,https://www.xeno-canto.org/142065,United States,Paul Driver,Empidonax alnorum_Alder Flycatcher,-68.9242,Not specified,06:00,Paul Driver,Creative Commons Attribution-NonCommercial-Sha...
17,4.5,no,aldfly,2 (stereo),2013-07-18,Not specified,315,XC144672.mp3,Not specified,Alder Flycatcher,...,144672,https://www.xeno-canto.org/144672,United States,Paul Driver,Empidonax alnorum_Alder Flycatcher,-68.389,Not specified,06:50,Paul Driver,Creative Commons Attribution-NonCommercial-Sha...
19,5.0,no,aldfly,2 (stereo),2013-07-18,Not specified,84,XC144675.mp3,Not specified,Alder Flycatcher,...,144675,https://www.xeno-canto.org/144675,United States,Paul Driver,Empidonax alnorum_Alder Flycatcher,-68.312,Not specified,06:20,Paul Driver,Creative Commons Attribution-NonCommercial-Sha...
30,4.5,no,aldfly,1 (mono),2014-05-25,Not specified,72,XC179600.mp3,Not specified,Alder Flycatcher,...,179600,https://www.xeno-canto.org/179600,United States,Jonathon Jongsma,Empidonax alnorum_Alder Flycatcher,-93.329,Not specified,06:00,Jonathon Jongsma,Creative Commons Attribution-ShareAlike 4.0
33,4.5,no,aldfly,2 (stereo),2014-05-25,Not specified,103,XC182735.mp3,Not specified,Alder Flycatcher,...,182735,https://www.xeno-canto.org/182735,Canada,Martin St-Michel,Empidonax alnorum_Alder Flycatcher,-72.5555,Not specified,06:53,Martin St-Michel,Creative Commons Attribution-NonCommercial-Sha...
34,4.5,no,aldfly,1 (mono),2014-06-13,level,127,XC188377.mp3,accelerating,Alder Flycatcher,...,188377,https://www.xeno-canto.org/188377,Canada,Richard E. Webster,Empidonax alnorum_Alder Flycatcher,-117.5883,0-3(s),07:55,Richard E. Webster,Creative Commons Attribution-NonCommercial-Sha...
35,4.5,no,aldfly,1 (mono),2014-06-14,level,22,XC188432.mp3,level,Alder Flycatcher,...,188432,https://www.xeno-canto.org/188432,Canada,Richard E. Webster,Empidonax alnorum_Alder Flycatcher,-120.0314,0-3(s),04:51,Richard E. Webster,Creative Commons Attribution-NonCommercial-Sha...
36,4.5,no,aldfly,1 (mono),2014-06-14,level,41,XC188434.mp3,level,Alder Flycatcher,...,188434,https://www.xeno-canto.org/188434,Canada,Richard E. Webster,Empidonax alnorum_Alder Flycatcher,-120.0314,0-3(s),04:52,Richard E. Webster,Creative Commons Attribution-NonCommercial-Sha...
37,4.5,no,aldfly,1 (mono),2014-06-15,level,120,XC188813.mp3,level,Alder Flycatcher,...,188813,https://www.xeno-canto.org/188813,Canada,Richard E. Webster,Empidonax alnorum_Alder Flycatcher,-123.1714,0-3(s),03:16,Richard E. Webster,Creative Commons Attribution-NonCommercial-Sha...


In [8]:
def getData(table):
    x_mfccs = []
    y_mfccs = []
    for idx,row in table.iterrows():
        print(idx)
        path = os.path.join("data/train_audio/",row["ebird_code"],row["filename"])
        y = DataLoader.readFromFile(path)
        if len(y)==0:
            continue
        mfcc = librosa.feature.mfcc(n_mfcc=50,y=y,sr=22050).T/1000
        
        try:
            rolled_mfcc = DataLoader.rollMFCC(mfcc, 215)
        except:
            continue
            
        x_mfcc = rolled_mfcc[:,:214,:]
        y_mfcc = rolled_mfcc[:,1:,:]
        x_mfccs.append(x_mfcc)
        y_mfccs.append(y_mfcc)

    x_train = np.concatenate(x_mfccs,axis=0)
    y_train = np.concatenate(y_mfccs,axis=0)
    x_train, y_train = shuffle(x_train, y_train)
    return x_train, y_train

In [9]:
x_train, y_train = getData(trainTable)
x_val, y_val = getData(valTable)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209


In [10]:
x_train.shape

(2388, 214, 50)

In [11]:
model = Sequential()

model.add(LSTM(20, activation="relu", return_sequences=True, input_shape=(None, 50)))
model.add(LSTM(10, activation="relu", return_sequences=True))
model.add(LSTM(20, activation="relu", return_sequences=True))
model.add(Dense(50, activation = None))

model.compile(loss=root_mean_squared_error,
              optimizer='adam')

history = model.fit(x_train, y_train, epochs=100, validation_data=(x_val, y_val))

Train on 2388 samples, validate on 57 samples
Epoch 1/100
2388/2388 [==============================] - 27s 11ms/step - loss: 0.0335 - val_loss: 0.0221
Epoch 2/100
2388/2388 [==============================] - 33s 14ms/step - loss: 0.0173 - val_loss: 0.0193
Epoch 3/100
2388/2388 [==============================] - 33s 14ms/step - loss: 0.0160 - val_loss: 0.0169
Epoch 4/100
2388/2388 [==============================] - 33s 14ms/step - loss: 0.0148 - val_loss: 0.0158
Epoch 5/100
2388/2388 [==============================] - 33s 14ms/step - loss: 0.0131 - val_loss: 0.0139
Epoch 6/100
2388/2388 [==============================] - 34s 14ms/step - loss: 0.0118 - val_loss: 0.0126
Epoch 7/100
2388/2388 [==============================] - 33s 14ms/step - loss: 0.0115 - val_loss: 0.0120
Epoch 8/100
2388/2388 [==============================] - 33s 14ms/step - loss: 0.0112 - val_loss: 0.0117
Epoch 9/100
2388/2388 [==============================] - 35s 15ms/step - loss: 0.0108 - val_loss: 0.0110
Epoch 10/

In [14]:
learnMap = history.history
plt.plot(learnMap["loss"])
plt.plot(learnMap["val_loss"])
plt.show()

In [90]:
def mse(a,b):
    e = a-b
    se = e*e
    mse = np.mean(se, axis=1)
    rmse = np.sqrt(mse)
    return rmse

def bgError(y):
    if len(y)==0:
        raise ValueError
    mfcc = librosa.feature.mfcc(n_mfcc=50,y=y,sr=22050).T/1000
    
    mfcc_pred = model.predict(np.array([mfcc]))[0,:,:]
    mfcc_pred = np.concatenate([mfcc[0:1,:],mfcc_pred[:-1,:]])
    
    return mse(mfcc, mfcc_pred)

def rollingMean(x, window, samples=None):
    x = [np.fabs(xx) for xx in x]
    window = int(window)
    xValArr = []

    if samples!=None:
        s = samples
        window = len(x)/samples
    else:
        s = int(len(x) / window)

    for i in range(s):
        val = np.mean(x[int(i * window): int(np.ceil((i + 1) * window))])
        xValArr.append(val)
    xValArr = np.array(xValArr)

    return xValArr

In [92]:
outFolder = "data/bgshock"
inFolder = "data/train_audio"
birds = ["aldfly"]

for bird in birds:
    os.makedirs(os.path.join(outFolder,bird), exist_ok=True)
    birdFolder = os.path.join(inFolder,bird)
    for file in os.listdir(birdFolder)[:2]:
        path = os.path.join(inFolder,bird,file)
        y = DataLoader.readFromFile(path)
        if len(y)==0:
            continue
        
        bgerr = bgError(y).flatten()
        rollsig = rollingMean(y, 512)

        plt.subplot(2,1,1)
        plt.plot(bgerr)

        plt.subplot(2,1,2)
        plt.plot(rollsig)
        plt.show()

[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
[0. 0. 0. ... 0. 0. 0.]
